In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from rnanorm import FPKM, TPM

from gtfparse import read_gtf
import re

In [3]:
df_gtf = read_gtf("gencode.v46.basic.annotation.gtf")

# Remove .1, .2, etc. from gene_id
def remove_version(s):
    return re.sub(r'\.\d+$', '', s)

gene_ids = df_gtf['gene_id'].to_list()
gene_ids = list(map(remove_version, gene_ids))
gene_ids = list(set(gene_ids))

INFO:root:Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_name', 'level', 'tag', 'transcript_id', 'transcript_type', 'transcript_name', 'transcript_support_level', 'havana_transcript', 'exon_number', 'exon_id', 'hgnc_id', 'havana_gene', 'ont', 'protein_id', 'ccdsid', 'artif_dupl']


In [5]:
meta_data = "/Users/xbh0403/Desktop/CSE_283/BENG-203-Project/datasets/burgos_dbgap/burgos_dbgap_metadata.xlsx"
meta_data = pd.read_excel(meta_data)

data = "/Users/xbh0403/Desktop/CSE_283/BENG-203-Project/datasets/burgos_dbgap/burgos_dbgap_counts.txt"
data = pd.read_csv(data, sep='\t')
data = data.T

# keep columns that are in the gene_ids
data = data.iloc[:, data.columns.isin(gene_ids)]

In [7]:
fpkm = FPKM(gtf="gencode.v46.basic.annotation.gtf").set_output(transform="pandas")
fpkm_normed_data = fpkm.fit_transform(data)

In [11]:
meta_data[["biospecimen_repository_sample_id"]]

,biospecimen_repository_sample_id
0,01_16_CSF
1,01_16_SER
2,01_18_CSF
3,01_18_SER
4,01_37_CSF
...,...
262,99_58_SER
263,99_60_CSF
264,99_60_SER
265,99_64_CSF
